In [1]:
from bs4 import BeautifulSoup
import requests
import os
import re
import pandas as pd

In [2]:
#try to create a new folder 'data'
try:
    os.mkdir('data')
    print("directory created")
except Exception as e: #'e' means for the same instruction - in this case, the directory 'data'
    print(f"Directory already exists {e}")
    pass

Directory already exists [WinError 183] Cannot create a file when that file already exists: 'data'


In [3]:
def save(t,n):
    #input the job page content as soup object t
    #convert t to string s
    s=t.prettify()
    
    #save the job page content in a text file number n as string
    with open(f"data/{n}.txt","w",encoding='utf-8') as g:
        g.write(s)
    return

In [4]:
def scrape(x,y):
    #to scrape the pages from page x to page y where y>=x.
    #it returns a list of soup objects of job page content
    
    #get the link
    URL = "https://hk.jobsdb.com/hk/search-jobs/data-scientist/"
    
    for p in range(x,y+1):
        #to make valid URL e.g https://hk.jobsdb.com/hk/search-jobs/data-science/1
        nextlink = URL+str(p)
        
        #if the link exists,get the resources
        if nextlink:
            html = requests.get(nextlink)
            soup = BeautifulSoup(html.text, "html.parser")

            #to find the links(directing to the job pages) from the resources
            #list l stores all the links
            l=[]
            for i in soup.find_all('a', href=True):
                if ('hk/en/job' in i['href'])&(i['href'][0]=="/"):
                    l.append("https://hk.jobsdb.com"+i['href'])

            #go to each job page by each link and get the content
            #result (list) stores the job contents
            result=[]
            for k in range(len(l)):
                i=l[k]
                #print("going to",i) #for checking only
                subhtml = requests.get(i)
                #print("success or not?",subhtml) #for checking only
                jobcontent = BeautifulSoup(subhtml.text, "html.parser")
                result.append(jobcontent)
                save(jobcontent,(p-1)*30+k+1)
                
        
        #if the link does not exist, it breaks the for loop.
        #i found that if the link does not exist, the DB website will
          #redirect you to page 1 automatically, which makes this part of code
          #does not work.
        else:
            print("Page",p,"does not exist.")
            break
    return result

In [34]:
#scrape the pages from 133 to 171
#scrape(133,171) we have done it.

section I: scrapping and saving as text
_______________________________________________
section II: gathering data

In [5]:
#to read the job content.txt:
def read(casenum):
    with open(f"data/{casenum}.txt","r",encoding='utf-8') as onlyread:
        text=onlyread.read()
    with open(f"data/{casenum}.txt","r",encoding='utf-8') as onlyread:
        line=onlyread.readlines()
    soup = BeautifulSoup(text, "html.parser")
    return text,line,soup

In [6]:
#transferred
def wordlocation(text,line,word,min_line,max_line):
    #it returns all the non-coding sentence(str) in a list after the target word
    #if max=min, a string is returned (but not a list)
    
    #text,line: pass variable text and line to the function
    #word: input the target word
    #min/max: the range of the non-coding sentence after the target word
    
    s=str(re.search(word,text))
    l=re.findall('\d+', s)
    if l==[]:
        #print("the word does not exist, return none") #for checking only
        return
    g=text[:int(l[1])+1].count("\n")

    p=1
    result=[]
    for j in range(g+1,g+60):
        if line[j].strip()[0]!="<":
            if ("Additional Information" in line[j].strip())==True:
                #print("already in add. info part") #for checking only
                if p==1:
                    return
                else:
                    if max_line==min_line:
                        return
                    else:
                        return result
            
            if line[j].strip()=="N/A":
                #print("they wrote N/A") #for checking only
                if p==1:
                    return
                else:
                    if max_line==min_line:
                        return
                    else:
                        return result
            
            if (p<=max_line)&(p>=min_line)&(len(line[j].strip())<800):
                if max_line==min_line:
                    return line[j].strip()
                else:
                    result.append(line[j].strip())
                    if p==max_line:
                        return result    
            p+=1

    #print("end of search. return none if nth is found. return a list if sth is found.") #for checking only
    if max_line==min_line:
        return
    else:
        if result==[]:
            return
        else:
            return result

In [18]:
#find post date. It returns date(str) in dd-mmm-yy (date,month,year format)
def date(text):
    s=str(re.search("Posted on",text))
    if s==None:
        s=str(re.search("Posted",text))
        if s==None:
            return
        else:
            return "05-Nov-21"
    else:
        datelist=re.findall('\d+', s)
        y=text[int(datelist[1])+1:int(datelist[1])+10].strip()
        if y[1]=="-":
            return '0'+y
        else:
            return y 

In [19]:
#find job title. It returns a string
def job(soup):
    return soup.find('h1').text.strip()

In [20]:
#it returns company name in string
def company(soup):
    longclass="sx2jih0 zcydq82q _18qlyvc0 _18qlyvcv _18qlyvc2 _1d0g9qk4 _18qlyvc9"
    return soup.find('span',class_=longclass).find_next('span',class_=longclass).text.strip()

In [21]:
#find location in string
def location(soup):
    y = soup.find('div',class_="sx2jih0 zcydq832").text
    if y==None:
        return
    else:
        return y.strip()

In [22]:
#find industry in string
def industry(text,line):
    s=str(re.search("Additional Company Information\n",text))
    l=re.findall('\d+', s)
    g=text[:int(l[1])+1].count("\n")
    
    p=0
    for j in range(g,g+20):
        if line[j].strip()[0]!="<":
            p+=1
        if p==2:
            return line[j].strip()
    #print("no industry found. return none") #for checking only
    return

In [23]:
#find years of experience
def exp(text,line):
    return wordlocation(text,line,"Years of Experience\n",1,1)

In [25]:
#find requirements in list
def req(text,line):
    word=["Requirements:","Requirements","Requirement","requirements:","requirements","requirement","REQUIREMENT"]
    
    for j in word:
        k = wordlocation(text,line,j,1,20)
        if (k!=[])|(k!=None):
            return k
    return

In [26]:
#find responsibilities in list
def res(text,line):
    word=["Responsibilities:","Responsibilities","responsibilities:","responsibilities","responsibility","Responsibility","RESPONSIBILIT"]
    
    for j in word:
        k = wordlocation(text,line,j,1,20)
        if (k!=[])|(k!=None):
            return k
    return

In [27]:
#find job description
def descri(text,line):
    return wordlocation(text,line,"Job Description",1,80)

In [28]:
#find the job case information,initial_casenum<=final_casenum
def jobinfo(initial_casenum,final_casenum):
    word=["Requirement","requirement","REQUIREMENT","Requirements","requirements","Requirement:","requirement:","Requirements:","requirements:",
      "Responsibility","responsibility","RESPONSIBILIT","Responsibilities","responsibilities","Responsibility:","responsibility:","Responsibilities:","responsibilities:"]

    for i in range(initial_casenum,final_casenum+1):
        text,line,soup=read(i)

        print("job case:",i)
        print("date:",date(text))
        print("job title:",job(soup))
        print("company name:",company(soup))
        print("location:",location(soup))
        print("industry:",industry(text,line))
        print("yrs of exp:",exp(text,line),"\n")
        for j in word:
            print(j,"=>",req(text,line,j),"\n")
        print("______________________The end________________________")
    return

In [47]:
###summary of functions:
#read(casenum): return text,line,soup
#date(text): return str(dd-mmm-yy) / none
#job(soup): return string
#company(soup): return string
#location(soup): return string / none
#industry(text,line): return string / none
#exp(text,line): return string / none
#req(text,line): return string / list / none
#res(text,line): return string / list / none
#descri(text,line): return list / none
#jobinfo(initial_casenum,final_casenum): print all the cases. return none.

section II: gathering data
_________________________________________
section III: finding abnormal job case (e.g no requirements) in the explanation file
_________________________________________
section IV: data frame

In [29]:
#to make a dataframe showing the requirement of job cases
#it returns a dataframe

def ourdf(initial_casenumber,final_casenumber):
    try:
        read(final_casenumber)
    except Exception as e: #'e' means for the same instruction - in this case, the directory 'data'
        print(f"the file {final_casenumber} doesn't exist ah bro~~ {e}")
        pass
    
    col=["date of post","job title","company name","location","industry","years of experience","requirement","responsibility","job description"]
    df = pd.DataFrame(columns=col,index=range(initial_casenumber,final_casenumber+1))

    for i in range(initial_casenumber,final_casenumber+1):
        text,line,soup=read(i)

        df.loc[i,"date of post"]=date(text)
        df.loc[i,"job title"]=job(soup)
        df.loc[i,"company name"]=company(soup)
        df.loc[i,"location"]=location(soup)
        df.loc[i,"industry"]=industry(text,line)
        df.loc[i,"years of experience"]=exp(text,line)
        df.loc[i,"requirement"]=req(text,line)
        df.loc[i,"responsibility"]=res(text,line)
        df.loc[i,"job description"]=descri(text,line)
        
        print("case number",i,": done")
    return df

In [78]:
#generate the dataframe using ourdf(initial_casenumber,final_casenumber)
df_kent = ourdf(3871,5130)
#df_(your name)=ourdf(your first case no,your final case no.)


#save dataframe as excel folder
df_kent.to_csv(path_or_buf="df_kent.csv")
#df_(your name).to_csv(path_or_buf="df_yourname.csv")


print("csv file saved!")

case number 3871 : done
case number 3872 : done
case number 3873 : done
case number 3874 : done
case number 3875 : done
case number 3876 : done
case number 3877 : done
case number 3878 : done
case number 3879 : done
case number 3880 : done
case number 3881 : done
case number 3882 : done
case number 3883 : done
case number 3884 : done
case number 3885 : done
case number 3886 : done
case number 3887 : done
case number 3888 : done
case number 3889 : done
case number 3890 : done
case number 3891 : done
case number 3892 : done
case number 3893 : done
case number 3894 : done
case number 3895 : done
case number 3896 : done
case number 3897 : done
case number 3898 : done
case number 3899 : done
case number 3900 : done
case number 3901 : done
case number 3902 : done
case number 3903 : done
case number 3904 : done
case number 3905 : done
case number 3906 : done
case number 3907 : done
case number 3908 : done
case number 3909 : done
case number 3910 : done
case number 3911 : done
case number 3912

case number 4214 : done
case number 4215 : done
case number 4216 : done
case number 4217 : done
case number 4218 : done
case number 4219 : done
case number 4220 : done
case number 4221 : done
case number 4222 : done
case number 4223 : done
case number 4224 : done
case number 4225 : done
case number 4226 : done
case number 4227 : done
case number 4228 : done
case number 4229 : done
case number 4230 : done
case number 4231 : done
case number 4232 : done
case number 4233 : done
case number 4234 : done
case number 4235 : done
case number 4236 : done
case number 4237 : done
case number 4238 : done
case number 4239 : done
case number 4240 : done
case number 4241 : done
case number 4242 : done
case number 4243 : done
case number 4244 : done
case number 4245 : done
case number 4246 : done
case number 4247 : done
case number 4248 : done
case number 4249 : done
case number 4250 : done
case number 4251 : done
case number 4252 : done
case number 4253 : done
case number 4254 : done
case number 4255

case number 4558 : done
case number 4559 : done
case number 4560 : done
case number 4561 : done
case number 4562 : done
case number 4563 : done
case number 4564 : done
case number 4565 : done
case number 4566 : done
case number 4567 : done
case number 4568 : done
case number 4569 : done
case number 4570 : done
case number 4571 : done
case number 4572 : done
case number 4573 : done
case number 4574 : done
case number 4575 : done
case number 4576 : done
case number 4577 : done
case number 4578 : done
case number 4579 : done
case number 4580 : done
case number 4581 : done
case number 4582 : done
case number 4583 : done
case number 4584 : done
case number 4585 : done
case number 4586 : done
case number 4587 : done
case number 4588 : done
case number 4589 : done
case number 4590 : done
case number 4591 : done
case number 4592 : done
case number 4593 : done
case number 4594 : done
case number 4595 : done
case number 4596 : done
case number 4597 : done
case number 4598 : done
case number 4599

case number 4902 : done
case number 4903 : done
case number 4904 : done
case number 4905 : done
case number 4906 : done
case number 4907 : done
case number 4908 : done
case number 4909 : done
case number 4910 : done
case number 4911 : done
case number 4912 : done
case number 4913 : done
case number 4914 : done
case number 4915 : done
case number 4916 : done
case number 4917 : done
case number 4918 : done
case number 4919 : done
case number 4920 : done
case number 4921 : done
case number 4922 : done
case number 4923 : done
case number 4924 : done
case number 4925 : done
case number 4926 : done
case number 4927 : done
case number 4928 : done
case number 4929 : done
case number 4930 : done
case number 4931 : done
case number 4932 : done
case number 4933 : done
case number 4934 : done
case number 4935 : done
case number 4936 : done
case number 4937 : done
case number 4938 : done
case number 4939 : done
case number 4940 : done
case number 4941 : done
case number 4942 : done
case number 4943

In [80]:
df_kent.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1260 entries, 3871 to 5130
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   date of post         1260 non-null   object
 1   job title            1260 non-null   object
 2   company name         1260 non-null   object
 3   location             1260 non-null   object
 4   industry             1260 non-null   object
 5   years of experience  853 non-null    object
 6   requirement          506 non-null    object
 7   responsibility       494 non-null    object
 8   job description      1260 non-null   object
dtypes: object(9)
memory usage: 88.7+ KB


In [30]:
df_daniel = ourdf(1291,2580)

case number 1291 : done
case number 1292 : done
case number 1293 : done
case number 1294 : done
case number 1295 : done
case number 1296 : done
case number 1297 : done
case number 1298 : done
case number 1299 : done
case number 1300 : done
case number 1301 : done
case number 1302 : done
case number 1303 : done
case number 1304 : done
case number 1305 : done
case number 1306 : done
case number 1307 : done
case number 1308 : done
case number 1309 : done
case number 1310 : done
case number 1311 : done
case number 1312 : done
case number 1313 : done
case number 1314 : done
case number 1315 : done
case number 1316 : done
case number 1317 : done
case number 1318 : done
case number 1319 : done
case number 1320 : done
case number 1321 : done
case number 1322 : done
case number 1323 : done
case number 1324 : done
case number 1325 : done
case number 1326 : done
case number 1327 : done
case number 1328 : done
case number 1329 : done
case number 1330 : done
case number 1331 : done
case number 1332

case number 1634 : done
case number 1635 : done
case number 1636 : done
case number 1637 : done
case number 1638 : done
case number 1639 : done
case number 1640 : done
case number 1641 : done
case number 1642 : done
case number 1643 : done
case number 1644 : done
case number 1645 : done
case number 1646 : done
case number 1647 : done
case number 1648 : done
case number 1649 : done
case number 1650 : done
case number 1651 : done
case number 1652 : done
case number 1653 : done
case number 1654 : done
case number 1655 : done
case number 1656 : done
case number 1657 : done
case number 1658 : done
case number 1659 : done
case number 1660 : done
case number 1661 : done
case number 1662 : done
case number 1663 : done
case number 1664 : done
case number 1665 : done
case number 1666 : done
case number 1667 : done
case number 1668 : done
case number 1669 : done
case number 1670 : done
case number 1671 : done
case number 1672 : done
case number 1673 : done
case number 1674 : done
case number 1675

case number 1976 : done
case number 1977 : done
case number 1978 : done
case number 1979 : done
case number 1980 : done
case number 1981 : done
case number 1982 : done
case number 1983 : done
case number 1984 : done
case number 1985 : done
case number 1986 : done
case number 1987 : done
case number 1988 : done
case number 1989 : done
case number 1990 : done
case number 1991 : done
case number 1992 : done
case number 1993 : done
case number 1994 : done
case number 1995 : done
case number 1996 : done
case number 1997 : done
case number 1998 : done
case number 1999 : done
case number 2000 : done
case number 2001 : done
case number 2002 : done
case number 2003 : done
case number 2004 : done
case number 2005 : done
case number 2006 : done
case number 2007 : done
case number 2008 : done
case number 2009 : done
case number 2010 : done
case number 2011 : done
case number 2012 : done
case number 2013 : done
case number 2014 : done
case number 2015 : done
case number 2016 : done
case number 2017

case number 2318 : done
case number 2319 : done
case number 2320 : done
case number 2321 : done
case number 2322 : done
case number 2323 : done
case number 2324 : done
case number 2325 : done
case number 2326 : done
case number 2327 : done
case number 2328 : done
case number 2329 : done
case number 2330 : done
case number 2331 : done
case number 2332 : done
case number 2333 : done
case number 2334 : done
case number 2335 : done
case number 2336 : done
case number 2337 : done
case number 2338 : done
case number 2339 : done
case number 2340 : done
case number 2341 : done
case number 2342 : done
case number 2343 : done
case number 2344 : done
case number 2345 : done
case number 2346 : done
case number 2347 : done
case number 2348 : done
case number 2349 : done
case number 2350 : done
case number 2351 : done
case number 2352 : done
case number 2353 : done
case number 2354 : done
case number 2355 : done
case number 2356 : done
case number 2357 : done
case number 2358 : done
case number 2359

In [34]:
df_daniel.to_csv(path_or_buf="df_daniel.csv")
#df_(your name).to_csv(path_or_buf="df_yourname.csv")


print("csv file saved!")

csv file saved!
